# Environnement

In [1]:
!pip install efficientnet_pytorch
!pip install tensorflow
# !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp37-cp37m-linux_x86_64.whl

In [2]:
import os
ROOT_DIR="/kaggle/working"
DATASET_ROOT_DIR="/kaggle/input/ufpr-alpr/UFPR-ALPR dataset"
CLASS_PLATE="PLATE"
CLASSES =[CLASS_PLATE]+ ["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]
CLASSES_IDS = {c:i for i, c in enumerate(CLASSES)}
NUM_CLASSES = len(CLASSES)

In [3]:
from time import strftime
import sys
import logging
def setup_logger():
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    a_logger = logging.getLogger()
    a_logger.setLevel("INFO")
    log_dir=os.path.join(ROOT_DIR,"logs","output_logs")
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    output_file_handler = logging.FileHandler(os.path.join(log_dir,strftime("log_%d_%m_%Y_%H_%M.log")))
    stdout_handler = logging.StreamHandler(sys.stdout)
    stdout_handler.setFormatter(formatter)
    a_logger.propagate=False
    a_logger.addHandler(output_file_handler)
    a_logger.addHandler(stdout_handler)
setup_logger()



In [4]:
import cv2
from matplotlib import pyplot as plt

def get_car_image(car_name):
    return os.path.join(constants.DATA_DIR,f"{car_name}.jpg")


def car_annotations(car_name):
    return os.path.join(constants.DATA_DIR, f"{car_name}.xml")

def collate_fn(batch):
    """
    To handle the data loading as different images may have different number
    of objects and to handle varying size tensors as well.
    """
    return tuple(zip(*batch))

BOX_COLOR = (255, 0, 0)  # Red
TEXT_COLOR = (255, 255, 255)  # White


def visualize_bbox(img, bbox, class_name, color=BOX_COLOR, thickness=2):
    """Visualizes a single bounding box on the image"""
    x_min, y_min, x_max,y_max=bbox

    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)

    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)
    cv2.putText(
        img,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.35,
        color=TEXT_COLOR,
        lineType=cv2.LINE_AA,
    )
    return img


def visualize(image, bboxes, category_ids, category_id_to_name):
    img = image.copy()
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)
    plt.show()

# Dataset

In [5]:
# the dataset class
import glob
import os
from enum import Enum
import albumentations as A
import cv2
import cv2.cv2
import numpy as np
from albumentations.pytorch import ToTensorV2
import torch
from PIL import Image
from torch.utils.data import Dataset
from torchvision.transforms import transforms

INPUT_SIZE = (600,600)
CROP_RATIO=1.0


class DatasetType(Enum):
    TRAIN = "training"
    VALID = "validation"
    TEST = "testing"
class CarPlateDataset(Dataset):
    def __init__(self, type: DatasetType = DatasetType.TRAIN):
        self.type = type
        self.dataset_dir = os.path.join(DATASET_ROOT_DIR, self.type.value)
        self.tracks_dict = {}

        self.tracks = []
        self.transforms=A.Compose([
            A.Resize(height=INPUT_SIZE[0], width=INPUT_SIZE[1], always_apply=True),
#             A.CenterCrop(height=INPUT_SIZE[0],width=INPUT_SIZE[1],p=1.0, always_apply=True),
            ToTensorV2(always_apply=True)
        ],
            bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']),
        )

        self.read_all_data()

    def __len__(self):
        return len(self.tracks)

    def read_all_data(self):
        """
        Real all data from the dataset

        :return:
        """
        for file in glob.glob(f'{self.dataset_dir}/**/*.txt', recursive=True):
            track = os.path.basename(file).split(".")[0]
            annotation_file = file
            image_file = file.replace(".txt", ".png")
            self.tracks_dict[track] = (image_file, annotation_file)
            self.tracks.append(track)

    def __getitem__(self, idx):
        track = self.tracks[idx]
        image_file, annotation_file = self.tracks_dict[track]
        with open(annotation_file, "r") as f :
            annotations = f.readlines()
        plate_id = annotations[6].split(":")[1].strip().replace("-", "")

        labels = [CLASSES_IDS[CLASS_PLATE]] + [CLASSES_IDS[c] for c in plate_id]
        image=cv2.imread(image_file)/255.0
        # Parsing annotations
        bboxes = []
        line_plate = annotations[7].split(":")[1].strip()
        plate_bbox = [float(x) for x in line_plate.split(" ")]
        plate_bbox = [plate_bbox[0], plate_bbox[1], plate_bbox[0]+plate_bbox[2], plate_bbox[1]+plate_bbox[3]]
        bboxes.append(plate_bbox)

        for line in annotations[8:]:
            content = line.split(":")[1].strip()
            bbox = [int(x) for x in content.split(" ")]
            bbox = [bbox[0], bbox[1], bbox[0]+bbox[2], bbox[1]+bbox[3]]
            bboxes.append(bbox)

        target=self.transforms(
            image=image,
            labels=np.array(labels),
            bboxes=np.array(bboxes)

        )
        target={"image":torch.as_tensor(target["image"],dtype=torch.float32),"labels":torch.as_tensor(target["labels"],dtype=torch.int64),
                "boxes":torch.as_tensor(target["bboxes"],dtype=torch.float32),
                "image_id":torch.as_tensor(idx,dtype=torch.float32)}
        return target["image"], target


### Network

In [6]:
import logging
import os

import torch

import torchvision
from efficientnet_pytorch import EfficientNet
from torch import nn
from torchvision.models.detection.anchor_utils import AnchorGenerator
from torchvision.models.detection.faster_rcnn import  FasterRCNN


use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

class ALPRNetwork(nn.Module):
    def __init__(self,model_name ="FasterRCNN",reset=False,load_best=False):
        super().__init__()
        self.model_name=model_name
        self.reset=reset
        self.load_best=load_best
        self.setup_checkpoints()
        self.setup_network()
        self.setup_checkpoints()


    #1. Setup Network archi
    def setup_network(self):
        # self.backbone = torchvision.models.__dict__["resnet18"](pretrained=True)
        # self.backbone= nn.Sequential(  *list(self.backbone.children())[:-1])  ## Remove the last layer of resnet as it is a classificatoin layer
        
        model_name = 'efficientnet-b7'
        model=EfficientNet.from_pretrained(model_name)
        conv_stem = torch.nn.Sequential(model._conv_stem)
        bn = torch.nn.Sequential(model._bn0)
        blocks = torch.nn.Sequential(*model._blocks)
        conv_head = torch.nn.Sequential(model._conv_head)
        #     conv_head.out_channels = 1280

        #Freezing some layers
        for p in conv_stem.parameters():p.requires_grad=False
        for child in list(blocks.children())[:-1]:
            for p in child.parameters():
                p.requires_grad=False


        self.backbone = torch.nn.Sequential(conv_stem, bn, blocks, conv_head)
        self.backbone.out_channels = 2560

        # Freezing some layers
        anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                           aspect_ratios=((0.5, 1.0, 2.0),))
        roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'], output_size=7, sampling_ratio=2)
        self.model = FasterRCNN(self.backbone,
                                 num_classes=NUM_CLASSES,
                                 rpn_anchor_generator=anchor_generator,
                                 box_roi_pool=roi_pooler)

    ##2. Model Saving/Loading
    def load_state(self):
        """
        Load model
        :param self:
        :return:
        """
        if self.load_best and os.path.exists(self.save_best_file):
            logging.info(f"Loading best model state : {self.save_file}")
            self.load_state_dict(torch.load(self.save_file, map_location=device))
            return

        if os.path.exists(self.save_file):
            logging.info(f"Loading model state : {self.save_file}")
            self.load_state_dict(torch.load(self.save_file, map_location=device))
    def save_state(self, best=False):
        if best:
            torch.save(self.state_dict(), self.save_best_file)
        else:
            torch.save(self.state_dict(), self.save_file)


    ##3. Setupping directories for weights /logs ... etc
    def setup_checkpoints(self):
        """
        Checking and creating directories for weights storage
        @return:
        """
        self.save_path = os.path.join(ROOT_DIR, 'zoos')
        self.model_dir = os.path.join(self.save_path, self.model_name)
        self.save_file = os.path.join(self.model_dir, f"{self.model_name}.pt")
        self.save_best_file = os.path.join(self.model_dir, f"{self.model_name}_best.pt")
        if not os.path.exists(self.model_dir):
            os.makedirs(self.model_dir)
        elif not self.reset:
            self.load_state()

    def forward(self, *args, **kwargs):
        return self.model(*args, **kwargs)


### Trainer 

In [7]:
import json
import logging
import os
import torch
from torch.utils.tensorboard import SummaryWriter
import tqdm
device="cuda"
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0


class Trainer:
    def __init__(self, network, optimizer, nb_epochs,reset=False):
        self.network = network
        self.optimizer = optimizer
        self.nb_epochs = nb_epochs

        self.tb_dir=os.path.join(ROOT_DIR, 'logs','tensorboard',network.model_name)
        self.info_file=os.path.join(self.tb_dir,'info.json')
        self.reset=reset
        if os.path.exists(self.info_file) and not self.reset:
            self.infos=json.load(open(self.info_file))


            ##Load learning rate:

        else:
            self.infos={'best_val_loss':float('inf'),'epoch':0,'train_itr':0,'lr':self.optimizer.param_groups[0]['lr'],

                   }
        self.summary_writer=SummaryWriter(self.tb_dir)



    def train(self,train_dataloader,valid_dataloader):
        self.network.to(device)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, 'min', factor=0.5, patience=5,verbose=True)
        logging.info('Training started on device : {}. lr={}'.format(device, self.optimizer.param_groups[0]['lr']))
        train_itr=self.infos['train_itr']

        for param_group in self.optimizer.param_groups:
            param_group['lr'] = self.infos['lr']

        start_epoch = self.infos['epoch'] if self.infos['epoch'] else 0
        self.nb_epochs=start_epoch+self.nb_epochs
        for epoch in range(start_epoch,self.nb_epochs):
            self.network.train()
            loss,loss_cf=self.train_epoch_loop(train_dataloader,epoch, train_itr)
            eval_loss,eval_clf_loss=self.validate_loop(valid_dataloader,epoch)
            scheduler.step(loss)
            train_itr=train_itr+len(train_dataloader)
            if eval_loss<self.infos['best_val_loss']:
                self.infos['best_val_loss']=eval_loss
            self.infos['epoch']=epoch
            self.infos['train_itr']=train_itr
            self.infos['lr']=self.optimizer.param_groups[0]['lr']
            json.dump(self.infos,open(self.info_file,'w'))
            logging.info('Epoch {}/{} : loss={:.4f} , loss_cf={:.4f} , eval_loss={:.4f} , eval_clf_loss={:.4f} , lr={:.6f}'.format(epoch,self.nb_epochs,loss,loss_cf,eval_loss,eval_clf_loss,self.infos['lr']))

    def train_epoch_loop(self,dataloader, epoch, train_itr):
        loss, loss_classification,loss_box_reg,loss_objectness=Averager(),Averager(),Averager(),Averager()
        for batch in tqdm.tqdm(dataloader, desc='Training epoch {}/{}'.format(epoch, self.nb_epochs)):
            train_itr += 1
            self.optimizer.zero_grad()

            images,targets=batch
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            loss_dict = self.network(images,targets)
            losses = sum(loss for loss in loss_dict.values())
            loss_value = losses.item()

            loss.send(loss_value)
            loss_classification.send(loss_dict['loss_classifier'].cpu().item())
            loss_box_reg.send(loss_dict['loss_box_reg'].cpu().item())
            loss_objectness.send(loss_dict['loss_objectness'].cpu().item())

            self.summary_writer.add_scalar('Train/loss', loss_value, train_itr)
            self.summary_writer.add_scalar('Train/loss_classification', loss_dict['loss_classifier'].cpu().item(),
                                           train_itr)
            self.summary_writer.add_scalar('Train/loss_box_reg', loss_dict['loss_box_reg'].cpu().item(), train_itr)
            self.summary_writer.add_scalar('Train/loss_objectness', loss_dict['loss_objectness'].cpu().item(),train_itr)
            losses.backward()
            self.optimizer.step()
        epoch_loss = loss.value
        epoch_loss_classification = loss_classification.value
        epoch_loss_box_reg = loss_box_reg.value
        epoch_loss_objectness = loss_objectness.value
        self.summary_writer.add_scalar('Train/epoch_loss', epoch_loss, epoch)
        self.summary_writer.add_scalar('Train/epoch_loss_classification', epoch_loss_classification, epoch)
        self.summary_writer.add_scalar('Train/epoch_loss_box_reg', epoch_loss_box_reg, epoch)
        self.summary_writer.add_scalar('Train/epoch_loss_objectness', epoch_loss_objectness, epoch)
        return epoch_loss, epoch_loss_classification


    def validate_loop(self,valid_dataloader,epoch):

        loss,loss_classification,loss_box_reg,loss_objectness=Averager(),Averager(),Averager(),Averager()
        with torch.no_grad():
            for batch in tqdm.tqdm(valid_dataloader,desc='Validating epoch {}'.format(epoch)):
                images,targets=batch
                images = list(image.to(device) for image in images)
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

                loss_dict=self.network(images,targets)
                losses = sum(loss for loss in loss_dict.values())
                loss_value = losses.item()
                loss.send(loss_value)
                loss_classification.send(loss_dict['loss_classifier'].cpu().item())
                loss_box_reg.send(loss_dict['loss_box_reg'].cpu().item())
                loss_objectness.send(loss_dict['loss_objectness'].cpu().item())

                self.summary_writer.add_scalar('Valid/loss',loss_value,epoch)
                self.summary_writer.add_scalar('Valid/loss_classification',loss_dict['loss_classifier'].cpu().item(),epoch)
                self.summary_writer.add_scalar('Valid/loss_box_reg',loss_dict['loss_box_reg'].cpu().item(),epoch)
                self.summary_writer.add_scalar('Valid/loss_objectness',loss_dict['loss_objectness'].cpu().item(),epoch)
        epoch_loss=loss.value
        epoch_loss_classification=loss_classification.value
        epoch_loss_box_reg=loss_box_reg.value
        epoch_loss_objectness=loss_objectness.value
        self.summary_writer.add_scalar('Valid/epoch_loss',epoch_loss,epoch)
        self.summary_writer.add_scalar('Valid/epoch_loss_classification',epoch_loss_classification,epoch)
        self.summary_writer.add_scalar('Valid/epoch_loss_box_reg',epoch_loss_box_reg,epoch)
        self.summary_writer.add_scalar('Valid/epoch_loss_objectness',epoch_loss_objectness,epoch)
        return epoch_loss,epoch_loss_classification




### Run Training

In [8]:

# %tensorflow_version 2.x
%load_ext tensorboard
%tensorboard --logdir logs/ 

In [ ]:
### from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.utils.data import DataLoader
reset=True
network=ALPRNetwork(reset=reset)
optimizer=Adam(lr=5e-4, params=[param for param in network.parameters() if param.requires_grad])
trainer=Trainer(network, optimizer,nb_epochs=200,reset=reset)
train_dataset,valid_dataset=CarPlateDataset(type=DatasetType.TRAIN),CarPlateDataset(type=DatasetType.VALID)
train_dataloader,valid_dataloader=DataLoader(train_dataset,batch_size=1,shuffle=True,collate_fn=collate_fn,num_workers=2),\
                                  DataLoader(valid_dataset,batch_size=1,shuffle=True,collate_fn=collate_fn,num_workers=2)
trainer.train(train_dataloader,valid_dataloader)

Loaded pretrained weights for efficientnet-b7
2022-05-27 17:24:19,280 - root - INFO - Training started on device : cuda. lr=0.0005


Validating epoch 0: 100%|██████████| 900/900 [02:48<00:00,  5.36it/s]

2022-05-27 17:41:22,355 - root - INFO - Epoch 0/200 : loss=0.5860 , loss_cf=0.2961 , eval_loss=0.1410 , eval_clf_loss=0.0925 , lr=0.000500



Validating epoch 1: 100%|██████████| 900/900 [02:46<00:00,  5.40it/s]

2022-05-27 17:58:26,155 - root - INFO - Epoch 1/200 : loss=0.0882 , loss_cf=0.0591 , eval_loss=0.2136 , eval_clf_loss=0.1478 , lr=0.000500



Validating epoch 2:  41%|████      | 365/900 [01:08<01:38,  5.41it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Validating epoch 3: 100%|██████████| 900/900 [02:41<00:00,  5.57it/s]

2022-05-27 18:32:08,186 - root - INFO - Epoch 3/200 : loss=0.0995 , loss_cf=0.0669 , eval_loss=0.2121 , eval_clf_loss=0.1355 , lr=0.000500



Training epoch 4/200:  78%|███████▊  | 1407/1800 [10:47<03:03,  2.14it/s]